In [2]:
import pandas as pd
import pickle

### Load Dataset

In [3]:
df_raw = pd.read_csv('./Churn_Modelling.csv')
df_raw.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
# Drop unnecessary features
df_raw.drop(labels=['RowNumber','CustomerId','Surname'],inplace=True,axis=1)
df_raw.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
# identify Feature types
feats_target = 'Exited'
feats_catg = [col for col in df_raw.columns if df_raw[col].dtypes == 'O']
feats_numr = [col for col in df_raw.columns if df_raw[col].dtypes != 'O' and col != feats_target]
print(feats_catg,feats_numr)

['Geography', 'Gender'] ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']


In [ ]:
# Train test split
from sklearn.model_selection import train_test_split, GridSearchCV

X = df_raw.drop(feats_target,axis=1)
Y = df_raw[feats_target]
x_train,x_vald,y_train,y_vald = train_test_split(X,Y,test_size=0.3,random_state=22)
x_vald,x_test,y_vald,y_test = train_test_split(x_vald,y_vald,test_size=0.5,random_state=22)

In [6]:
x_test

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
5880,537,France,Male,28,0,88963.31,2,1,1,189839.93
9903,606,France,Female,36,10,0.00,2,0,1,155641.46
486,641,France,Male,37,7,0.00,2,1,0,75248.30
1171,693,Germany,Male,40,0,120711.73,1,0,0,27345.18
2111,642,France,Male,25,7,0.00,2,1,0,102083.78
...,...,...,...,...,...,...,...,...,...,...
7395,721,Germany,Female,45,7,138523.20,1,0,0,59604.45
2640,625,Spain,Female,31,8,0.00,2,1,0,151843.54
1943,728,Germany,Male,39,6,152182.83,1,0,0,161203.60
6288,689,France,Male,40,8,160272.27,1,1,0,49656.24


In [7]:
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer

# Create a preprocessing pipeline object
ppln_prpc = ColumnTransformer(transformers=[
    ('Categorical',OneHotEncoder(sparse_output=False,drop='if_binary'),feats_catg),
    ('Numerical',StandardScaler(),feats_numr),
    # ('Numerical',MinMaxScaler(copy=False),feats_numr),
],remainder="passthrough", verbose_feature_names_out=False, force_int_remainder_cols=False, n_jobs=-1).set_output(transform='pandas')

ppln_prpc   

,transformers,"[('Categorical', ...), ('Numerical', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,-1
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False
,force_int_remainder_cols,False
,categories,'auto'
,drop,'if_binary'
,sparse_output,False


In [ ]:
# Data Transformation
x_train_tf = ppln_prpc.fit_transform(x_train)
x_vald_tf = ppln_prpc.transform(x_vald)
x_test_tf = ppln_prpc.transform(x_test)
x_train_tf

with open('bc_ppln.pkl','wb') as file:
    pickle.dump(ppln_prpc,file)

/home/who/Documents/Coding/Courses/Krish Nayak/Full_ML_DL_NLP/Study_Customer-Churn/.venv/lib/python3.12/site-packages/sklearn/compose/_column_transformer.py:975: FutureWarning: The parameter `force_int_remainder_cols` is deprecated and will be removed in 1.9. It has no effect. Leave it to its default value to avoid this warning.
  warnings.warn(


In [1]:
x_train_tf.columns

NameError: name 'x_train_tf' is not defined

In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from datetime import datetime

2025-08-08 10:12:23.721186: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-08 10:12:24.012457: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-08 10:12:24.142799: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754628144.433813    2591 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754628144.494457    2591 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754628145.079511    2591 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [10]:
# Setup ANN model

model = Sequential([
    Dense(64,activation='relu',input_shape=(x_train_tf.shape[1],)),
    Dense(64,activation='relu'),
    Dense(32,activation='relu'),
    Dense(16,activation='relu'),
    Dense(1,activation='sigmoid'),
])

/home/who/Documents/Coding/Courses/Krish Nayak/Full_ML_DL_NLP/Study_Customer-Churn/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-08-08 10:12:33.917524: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [11]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,617 (29.75 KB)

 Trainable params: 7,617 (29.75 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
optm = tf.keras.optimizers.Adam(learning_rate=0.01)
loss = tf.keras.losses.BinaryCrossentropy()
mtrc = [tf.keras.metrics.Accuracy()]

In [13]:
# model compilation

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
# model.compile(optimizer=optm,loss=loss,metrics=mtrc)


In [14]:
# Setup Tensorboard
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard


log_dir = f'logs/fit/{datetime.now().strftime('%Y-%m-%d_%H-%S-%M')}'
tf_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)


In [15]:
# Setup Earlystop
es_callback = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)


In [16]:
# Model Training
history = model.fit(x_train_tf,y_train,validation_data=(x_vald_tf,y_vald),epochs=100,callbacks=[tf_callback,es_callback])



Epoch 1/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.8146 - loss: 0.4281 - val_accuracy: 0.8487 - val_loss: 0.3709
Epoch 2/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8554 - loss: 0.3493 - val_accuracy: 0.8427 - val_loss: 0.3667
Epoch 3/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8599 - loss: 0.3378 - val_accuracy: 0.8427 - val_loss: 0.3733
Epoch 4/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8590 - loss: 0.3324 - val_accuracy: 0.8507 - val_loss: 0.3645
Epoch 5/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8620 - loss: 0.3277 - val_accuracy: 0.8400 - val_loss: 0.3764
Epoch 6/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8649 - loss: 0.3253 - val_accuracy: 0.8447 - val_loss: 0.3658
Epoch 7/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8673 - loss: 0.3207 - val_accuracy: 0.8447 - val_loss: 0.3678
Epoch 8/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8633 - loss: 0.3192 - val_accu

In [ ]:
model.save('bc_model.keras')

In [18]:
# # Tensorboard

# %load_ext tensorboard
# %tensorboard --logdir logs/fit

In [ ]:
# predict using model
from tensorflow.keras.models import load_model

model = load_model('bc_model.keras')


In [20]:
y_train_pred = model.predict(x_train_tf)
y_train_pred = pd.Series(y_train_pred.flatten())
y_train_pred = (y_train_pred > 0.7).astype(int)
# y_train_pred

219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [21]:
y_vald_pred = model.predict(x_vald_tf)
y_vald_pred = pd.Series(y_vald_pred.flatten())
y_vald_pred = (y_vald_pred > 0.7).astype(int)
# y_vald_pred

47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [22]:
y_test_pred = model.predict(x_test_tf)
y_test_pred = pd.Series(y_test_pred.flatten())
y_test_pred = (y_test_pred > 0.7).astype(int)
# y_test_pred

47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


In [23]:
from sklearn.metrics import accuracy_score

print(f'Accuracy_Train: {accuracy_score(y_train,y_train_pred)}')
print(f'Accuracy_Vald: {accuracy_score(y_vald,y_vald_pred)}')
print(f'Accuracy_Test: {accuracy_score(y_test,y_test_pred)}')

Accuracy_Train: 0.855
Accuracy_Vald: 0.8346666666666667
Accuracy_Test: 0.856
